In [1]:
import torch
import argparse
import torch.nn as nn
import torch.utils.data as Data
import torch.backends.cudnn as cudnn
import pandas as pd
from scipy.io import savemat
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from torch import optim
from model import DSNet
from utils import AvgrageMeter, accuracy, output_metric, NonZeroClipper, print_args
from dataset import prepare_dataset
import numpy as np
import time
import os
import wandb
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch


In [14]:
parser = argparse.ArgumentParser("HSI")
parser.add_argument('--fix_random', action='store_true', default=True, help='fix randomness')
parser.add_argument('--gpu_id', default='0', help='gpu id')
parser.add_argument('--seed', type=int, default=0, help='number of seed')
parser.add_argument('--dataset', choices=['Indian', 'Berlin', 'Augsburg','GS','BS'], default='BS', help='dataset to use')
parser.add_argument('--flag_test', choices=['test', 'train'], default='train', help='testing mark')
parser.add_argument('--model_name', choices=['conv2d_unmix'], default='conv2d_unmix', help='DSNet')
parser.add_argument('--batch_size', type=int, default=128, help='number of batch size')
parser.add_argument('--test_freq', type=int, default=10, help='number of evaluation')
parser.add_argument('--patches', type=int, default=5, help='number of patches')
parser.add_argument('--epoches', type=int, default=100, help='epoch number')
parser.add_argument('--inference_model', type=str, default='./results/BS_06_conv2d_unmix_p5_97.02_epoch20.pkl', help='inference model')
parser.add_argument('--learning_rate', type=float, default=1e-4, help='learning rate')
parser.add_argument('--gamma', type=float, default=0.9, help='gamma')
parser.add_argument('--weight_decay', type=float, default=0, help='weight_decay')
args = parser.parse_args(args=[])

In [15]:
def train_epoch(model, train_loader, criterion, optimizer,lambda_value=0.5):
    model.train()
    objs = AvgrageMeter()
    top1 = AvgrageMeter()
    tar = np.array([])
    pre = np.array([])

    eps = 1e-7
    # add tqdm for progress bar
    for batch_idx, (batch_data, batch_target) in enumerate(train_loader):
        # Ensure batch_data is a Tensor
        if isinstance(batch_data, np.ndarray):
            batch_data = torch.from_numpy(batch_data)

        batch_data = batch_data.cuda()
        batch_target = batch_target.cuda()
        batch_target = batch_target.long().cuda()

        # 입력 차원을 조정합니다. (batch_size, channels, sequence_length)
        if batch_data.dim() == 2:
            batch_data = batch_data.unsqueeze(-1)  # (batch_size, channels) -> (batch_size, channels, 1)

        optimizer.zero_grad()
        if 'unmix' in args.model_name:
            re_unmix_nonlinear, re_unmix, batch_pred = model(batch_data)

            band = re_unmix.shape[1]//2  # 2 represents the number of layer
            output_linear = re_unmix[:,0:band] + re_unmix[:,band:band*2]
            re_unmix = re_unmix_nonlinear + output_linear

            cosine_similarity = torch.sum(batch_data * re_unmix, dim=1) / (
                torch.norm(re_unmix, dim=1, p=2) * torch.norm(batch_data, dim=1, p=2) + eps
            )
            # 범위 클램핑
            cosine_similarity = torch.clamp(cosine_similarity, -0.999, 0.999)

            sad_loss = torch.mean(torch.acos(cosine_similarity))
            loss = lambda_value * sad_loss + (1 - lambda_value) * criterion(batch_pred, batch_target)
            
        else:
            batch_pred = model(batch_data)
            loss = criterion(batch_pred, batch_target)

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"NaN or Inf detected in loss at batch {batch_idx}. Skipping this batch.")
            continue
                
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        prec1, t, p = accuracy(batch_pred, batch_target, topk=(1,))
        n = batch_data.shape[0]
        objs.update(loss.item(), n)  # loss.item()으로 접근
        top1.update(prec1[0].item(), n)  # prec1[0].item()으로 접근

        tar = np.append(tar, t.data.cpu().numpy())
        pre = np.append(pre, p.data.cpu().numpy())
    return top1.avg, objs.avg, tar, pre

In [16]:
def valid_epoch(model, valid_loader, criterion, optimizer):
    objs = AvgrageMeter()
    top1 = AvgrageMeter()
    tar = np.array([])
    pre = np.array([])
    for batch_idx, (batch_data, batch_target) in enumerate(valid_loader):
        batch_data = batch_data.cuda()
        batch_target = batch_target.cuda()

        if 'unmix' in args.model_name:
            re_unmix_nonlinear, re_unmix, batch_pred = model(batch_data)

            band = re_unmix.shape[1]//2
            output_linear = re_unmix[:,0:band] + re_unmix[:,band:band*2]
            re_unmix = re_unmix_nonlinear + output_linear

            sad_loss = torch.mean(torch.acos(torch.sum(batch_data * re_unmix, dim=1)/
                        (torch.norm(re_unmix, dim=1, p=2) * torch.norm(batch_data, dim=1, p=2))))
            loss = criterion(batch_pred, batch_target) + sad_loss
        else:
            batch_pred = model(batch_data)
            loss = criterion(batch_pred, batch_target)

        prec1, t, p = accuracy(batch_pred, batch_target, topk=(1,))
        n = batch_data.shape[0]
        objs.update(loss.data, n)
        top1.update(prec1[0].data, n)
        tar = np.append(tar, t.data.cpu().numpy())
        pre = np.append(pre, p.data.cpu().numpy())
    return tar, pre

In [17]:
def test_epoch(model, test_loader):
    pre = np.array([])
    for batch_idx, (batch_data, batch_target) in enumerate(test_loader):
        batch_data = batch_data.cuda()
        batch_target = batch_target.cuda()

        if 'unmix' in args.model_name:
            re_unmix_nonlinear, re_unmix, batch_pred = model(batch_data)
        else:
            batch_pred = model(batch_data)

        _, pred = batch_pred.topk(1, 1, True, True)
        pp = pred.squeeze()
        pre = np.append(pre, pp.data.cpu().numpy())

    return pre

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu_id)
seeds = [20250401, 20250402,20250403,20250404,20250405,20250406,20250407,20250408,20250409,20250410]
if args.fix_random:
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
else:
    cudnn.benchmark = True

for i in tqdm(seeds):
    ## prepare dataset
    label_train_loader, label_test_loader, label_true_loader, band, height, width, num_classes, label, total_pos_true = prepare_dataset(args,train_ratio=0.03)
    # create model
    if args.model_name == 'conv2d_unmix':
        model = DSNet(band, num_classes, args.patches, args.model_name)
    else:
        raise KeyError("{} model is unknown.".format(args.model_name))
    model = model.cuda()
    print("Model Name: {}".format(args.model_name))
    
    criterion = nn.CrossEntropyLoss().cuda()
    # Set the optimizer
    if 'unmix' in args.model_name:
        params = map(id, model.unmix_decoder.parameters())
        ignored_params = list(set(params))
        base_params = filter(lambda p: id(p) not in ignored_params, model.parameters())
        optimizer = torch.optim.Adam([{'params': base_params},{'params': model.unmix_decoder.parameters(), 'lr': 3e-4}],
                                    lr = args.learning_rate, weight_decay = args.weight_decay)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    
    apply_nonegative = NonZeroClipper()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.epoches//10, gamma=args.gamma)
    #-------------------------------------------------------------------------------
    print("start training")
    tic = time.time()
    min_val_obj, best_OA = 0.5, 0
    for epoch in range(args.epoches):
        scheduler.step()

        # train model
        model.train()
        train_acc, train_obj, tar_t, pre_t = train_epoch(model, label_train_loader, criterion, optimizer)
        OA1, AA_mean1, Kappa1, AA1, precision1, recall1, f1_1 = output_metric(tar_t, pre_t)
        print("Epoch: {:03d} train_loss: {:.4f} train_acc: {:.4f} Precision: {:.4f} Recall: {:.4f} F1-Score: {:.4f}".format(epoch+1, train_obj, train_acc, precision1, recall1, f1_1))

        if 'unmix' in args.model_name: # regularize unmix decoder
            model.unmix_decoder.apply(apply_nonegative)

        if (epoch % args.test_freq == 0) | (epoch == args.epoches - 1):
            model.eval()
            tar_v, pre_v = valid_epoch(model, label_test_loader, criterion, optimizer)
            OA2, AA_mean2, Kappa2, AA2, precision2, recall2, f1_2 = output_metric(tar_v, pre_v)
            print("OA: {:.4f} AA: {:.4f} Kappa: {:.4f} Precision: {:.4f} Recall: {:.4f} F1-Score: {:.4f}".format(OA2, AA_mean2, Kappa2, precision2, recall2, f1_2))
            print("*************************")

            if f1_2 > min_val_obj and epoch > 10:
                model_save_path = os.path.join('./results/', args.dataset+'_07_ratio_'+str(i)+'_'+args.model_name+'_p'+str(args.patches)+
                                                '_'+str(round(OA2*100, 2))+'_epoch'+str(epoch)+"_seed_"+str(i)+'.pkl')
                torch.save(model.state_dict(), model_save_path)

                min_val_obj = OA2
                best_epoch = epoch
                best_OA = OA2
                best_AA = AA_mean2
                best_Kappa = Kappa2
                best_each_AA = AA2

    toc = time.time()
    print("Running Time: {:.2f}".format(toc-tic))
    print("**************************************************")
    if best_OA == 0:
        model_save_path = os.path.join('./results/', args.dataset+'_07_ratio_'+str(i)+'_'+args.model_name+'_p'+str(args.patches)+
                                        '_'+str(round(OA2*100, 2))+'_epoch'+str(epoch)+"_seed_"+str(i)+'_.pkl')
        torch.save(model.state_dict(), model_save_path)
        wandb.save(model_save_path)

    model.eval()
    pre_u = test_epoch(model, label_true_loader)

    prediction_matrix = np.zeros((height, width), dtype=float)
    for i in range(total_pos_true.shape[0]):
        prediction_matrix[total_pos_true[i,0], total_pos_true[i,1]] = pre_u[i] + 1

    # 예측 및 레이블 데이터 플래트닝 및 타입 변환
    flattened_label = label.flatten().astype(int)
    flattened_prediction = prediction_matrix.flatten().astype(int)
    print(f'flattened_label shape: {flattened_label.shape}')
    print(f'flattened_prediction shape: {flattened_prediction.shape}')
    flattened_label[flattened_label==2]=0
    flattened_prediction[flattened_prediction==2]=0

    # F1 score, precision, and recall calculation
    try:
        f1 = f1_score(flattened_label, flattened_prediction, average='binary')
        precision = precision_score(flattened_label, flattened_prediction, average='binary')
        recall = recall_score(flattened_label, flattened_prediction, average='binary')
    except ValueError as e:
        print(f"Error: {e}. Switching to 'macro' average.")
        f1 = f1_score(flattened_label, flattened_prediction, average='macro')
        precision = precision_score(flattened_label, flattened_prediction, average='macro')
        recall = recall_score(flattened_label, flattened_prediction, average='macro')

    conf_matrix = confusion_matrix(flattened_label, flattened_prediction)
    print("**************************************************")
    print(f'F1 score: {f1}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'max_label: {np.max(flattened_label)} max_pred: {np.max(flattened_prediction)}')
    print(f'Accuracy: {np.sum(flattened_label == flattened_prediction) / flattened_label.size}')
    print("Confusion Matrix:")
    print(conf_matrix)
    print("**************************************************")
    print("Parameter:") 
    print_args(vars(args))

  0%|          | 0/10 [00:00<?, ?it/s]

Train data change to the ratio of train samples: 0.03
height=2715,width=1843,band=150
**************************************************
patch is : 5
mirror_image shape : [2719,1847,150]
**************************************************
x_train shape = (116561, 5, 5, 150), type = float32
x_test  shape = (3768786, 5, 5, 150), type = float32
x_true  shape = (5003745, 5, 5, 150), type = float32
**************************************************
y_train: shape = (116561,) ,type = int64
y_test: shape = (3768786,) ,type = int64
y_true: shape = (5003745,) ,type = int64
**************************************************
Model Name: conv2d_unmix
start training


/home1/jmt30269/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


akk: 5764, gt: 110797
Epoch: 001 train_loss: 1.0209 train_acc: 94.5711 Precision: 0.0330 Recall: 0.2013 F1-Score: 0.0566
akk: 49244, gt: 3719542
OA: 0.9845 AA: 0.6685 Kappa: 0.2583 Precision: 0.2152 Recall: 0.3473 F1-Score: 0.2658
*************************
akk: 3274, gt: 113287
Epoch: 002 train_loss: 0.1851 train_acc: 96.7245 Precision: 0.0611 Recall: 0.2119 F1-Score: 0.0948
akk: 1174, gt: 115387
Epoch: 003 train_loss: 0.0732 train_acc: 98.4386 Precision: 0.1269 Recall: 0.1578 F1-Score: 0.1407
akk: 471, gt: 116090
Epoch: 004 train_loss: 0.0512 train_acc: 99.1618 Precision: 0.4650 Recall: 0.2320 F1-Score: 0.3095
akk: 473, gt: 116088
Epoch: 005 train_loss: 0.0530 train_acc: 99.1944 Precision: 0.5053 Recall: 0.2532 F1-Score: 0.3373
akk: 556, gt: 116005
Epoch: 006 train_loss: 0.0421 train_acc: 99.3137 Precision: 0.6295 Recall: 0.3708 F1-Score: 0.4667
akk: 556, gt: 116005
Epoch: 007 train_loss: 0.0410 train_acc: 99.3274 Precision: 0.6439 Recall: 0.3792 F1-Score: 0.4773
akk: 582, gt: 115979


 10%|█         | 1/10 [53:07<7:58:11, 3187.95s/it]

...

**************************************************
F1 score: 0.6863884982028442
Precision: 0.6750791552672835
Recall: 0.6980832194284624
max_label: 1 max_pred: 1
Accuracy: 0.9959894039364516
Confusion Matrix:
[[4961716   10570]
 [   9498   21961]]
**************************************************
Parameter:
fix_random: True
gpu_id: 0
seed: 0
dataset: BS
flag_test: train
model_name: conv2d_unmix
batch_size: 128
test_freq: 10
patches: 5
epoches: 100
inference_model: ./results/BS_06_conv2d_unmix_p5_97.02_epoch20.pkl
learning_rate: 0.0001
gamma: 0.9
weight_decay: 0
Train data change to the ratio of train samples: 0.03
height=2715,width=1843,band=150
**************************************************
patch is : 5
mirror_image shape : [2719,1847,150]
**************************************************
x_train shape = (116561, 5, 5, 150), type = float32
x_test  shape = (3768786, 5, 5, 150), type = float32
x_true  shape = (5003745, 5, 5, 150), type = float32
***********************************

/home1/jmt30269/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


akk: 5849, gt: 110712
Epoch: 001 train_loss: 0.7264 train_acc: 94.4741 Precision: 0.0301 Recall: 0.1864 F1-Score: 0.0518
akk: 1358780, gt: 2410006
OA: 0.6461 AA: 0.7775 Kappa: 0.0246 Precision: 0.0205 Recall: 0.9111 F1-Score: 0.0400
*************************
akk: 3121, gt: 113440
Epoch: 002 train_loss: 0.1633 train_acc: 96.8935 Precision: 0.0711 Recall: 0.2352 F1-Score: 0.1092
akk: 2389, gt: 114172
Epoch: 003 train_loss: 0.0986 train_acc: 97.5815 Precision: 0.1076 Recall: 0.2722 F1-Score: 0.1542
akk: 1671, gt: 114890
Epoch: 004 train_loss: 0.0784 train_acc: 98.1580 Precision: 0.1400 Recall: 0.2479 F1-Score: 0.1790
akk: 1438, gt: 115123
Epoch: 005 train_loss: 0.0662 train_acc: 98.3923 Precision: 0.1766 Recall: 0.2691 F1-Score: 0.2133
akk: 650, gt: 115911
Epoch: 006 train_loss: 0.0519 train_acc: 99.0065 Precision: 0.3354 Recall: 0.2309 F1-Score: 0.2735
akk: 522, gt: 116039
Epoch: 007 train_loss: 0.0428 train_acc: 99.3137 Precision: 0.6379 Recall: 0.3528 F1-Score: 0.4543
akk: 545, gt: 116

100%|██████████| 10/10 [9:11:51<00:00, 3311.17s/it]

**************************************************
F1 score: 0.7037957330134521
Precision: 0.7634964306960142
Recall: 0.6527543787151531
max_label: 1 max_pred: 1
Accuracy: 0.996545587355071
Confusion Matrix:
[[4965925    6361]
 [  10924   20535]]
**************************************************
Parameter:
fix_random: True
gpu_id: 0
seed: 0
dataset: BS
flag_test: train
model_name: conv2d_unmix
batch_size: 128
test_freq: 10
patches: 5
epoches: 100
inference_model: ./results/BS_06_conv2d_unmix_p5_97.02_epoch20.pkl
learning_rate: 0.0001
gamma: 0.9
weight_decay: 0


## test

In [22]:
import os
base_path = '/home1/jmt30269/DSNet/results/'
list_dir = os.listdir(base_path)
list_dir.sort()

filtered_files = [f for f in list_dir if 'BS_07_ratio_202504' in f]

In [23]:
filtered_files

['BS_07_ratio_20250401_conv2d_unmix_p5_99.51_epoch20_seed_20250401.pkl',
 'BS_07_ratio_20250402_conv2d_unmix_p5_99.44_epoch40_seed_20250402.pkl',
 'BS_07_ratio_20250403_conv2d_unmix_p5_99.43_epoch20_seed_20250403.pkl',
 'BS_07_ratio_20250404_conv2d_unmix_p5_99.41_epoch30_seed_20250404.pkl',
 'BS_07_ratio_20250405_conv2d_unmix_p5_99.29_epoch20_seed_20250405.pkl',
 'BS_07_ratio_20250406_conv2d_unmix_p5_99.51_epoch20_seed_20250406.pkl',
 'BS_07_ratio_20250407_conv2d_unmix_p5_99.4_epoch20_seed_20250407.pkl',
 'BS_07_ratio_20250408_conv2d_unmix_p5_99.52_epoch20_seed_20250408.pkl',
 'BS_07_ratio_20250409_conv2d_unmix_p5_99.5_epoch20_seed_20250409.pkl',
 'BS_07_ratio_20250410_conv2d_unmix_p5_99.11_epoch20_seed_20250410.pkl']

In [24]:
import torch.nn.functional as F
def test_epoch(model, test_loader):
    pre = np.array([])  # 예측 레이블 저장
    probs = []  # 각 클래스별 확률 저장

    for batch_idx, (batch_data, batch_target) in enumerate(test_loader):
        batch_data = batch_data.cuda()
        batch_target = batch_target.cuda()

        if 'unmix' in args.model_name:
            re_unmix_nonlinear, re_unmix, batch_pred = model(batch_data)
        else:
            batch_pred = model(batch_data)

        # Softmax 적용하여 확률 계산
        batch_probs = F.softmax(batch_pred, dim=1)  # (배치 크기, 클래스 수)

        _, pred = batch_pred.topk(1, 1, True, True)  # 최상위 클래스 예측
        pp = pred.squeeze()

        pre = np.append(pre, pp.data.cpu().numpy())
        probs.append(batch_probs.data.cpu().numpy())  # 확률 저장

    return pre, np.vstack(probs)  # 예측 레이블과 확률 반환

In [25]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu_id)
print(os.environ["CUDA_VISIBLE_DEVICES"])
if args.fix_random:
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
else:
    cudnn.benchmark = True

## prepare dataset
label_train_loader, label_test_loader, label_true_loader, band, height, width, num_classes, label, total_pos_true = prepare_dataset(args,train_ratio=0.015)
# create model
if args.model_name == 'conv2d_unmix':
    model = DSNet(band, num_classes, args.patches, args.model_name)
else:
    raise KeyError("{} model is unknown.".format(args.model_name))
model = model.cuda()
print("Model Name: {}".format(args.model_name))

0
Train data change to the ratio of train samples: 0.015
height=2715,width=1843,band=150
**************************************************
patch is : 5
mirror_image shape : [2719,1847,150]
**************************************************
x_train shape = (58281, 5, 5, 150), type = float32
x_test  shape = (3827066, 5, 5, 150), type = float32
x_true  shape = (5003745, 5, 5, 150), type = float32
**************************************************
y_train: shape = (58281,) ,type = int64
y_test: shape = (3827066,) ,type = int64
y_true: shape = (5003745,) ,type = int64
**************************************************
Model Name: conv2d_unmix


In [26]:
# criterion
# create model
import re
for run in filtered_files:
    # 정규표현식으로 8자리 숫자 추출
    match = re.search(r'202504\d{2}', run)
    if match:
        date_str = match.group()
        print(date_str)
    if args.model_name == 'conv2d_unmix':
        model = DSNet(band, num_classes, args.patches, args.model_name)
    else:
        raise KeyError("{} model is unknown.".format(args.model_name))
    model = model.cuda()
    run=os.path.join(base_path,run)
    from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
    criterion = nn.CrossEntropyLoss().cuda()
    # Set the optimizer
    if 'unmix' in args.model_name:
        params = map(id, model.unmix_decoder.parameters())
        ignored_params = list(set(params))
        base_params = filter(lambda p: id(p) not in ignored_params, model.parameters())
        optimizer = torch.optim.Adam([{'params': base_params},{'params': model.unmix_decoder.parameters(), 'lr': 3e-4}],
                                    lr = args.learning_rate, weight_decay = args.weight_decay)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    
    apply_nonegative = NonZeroClipper()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.epoches//10, gamma=args.gamma)
    
    #-------------------------------------------------------------------------------
    model.eval()
    model.load_state_dict(torch.load(run))
    pre_u,prob = test_epoch(model, label_true_loader)
    
    prediction_matrix = np.zeros((height, width), dtype=float)
    for i in range(total_pos_true.shape[0]):
        prediction_matrix[total_pos_true[i,0], total_pos_true[i,1]] = pre_u[i] + 1
    # savemat('07_matrix.mat',{'P':prediction_matrix, 'label':label})
    
    # 예측 및 레이블 데이터 플래트닝 및 타입 변환
    flattened_label = label.flatten().astype(int)
    flattened_prediction = prediction_matrix.flatten().astype(int)
    print(f'flattened_label shape: {flattened_label.shape}')
    print(f'flattened_prediction shape: {flattened_prediction.shape}')
    data111 = list(zip(flattened_label, prob, flattened_prediction))
    # 문자열에서 대괄호와 공백 제거 후 숫자 추출
    df=pd.DataFrame(data111,columns=['gt','prob','pred'])
    back=[]
    akk=[]
    for i in df['prob']:
        akk.append(i[0])
        back.append(i[1])
    df['prob_akk']=akk
    df['prob_back']=back
    # prob 칼럼 삭제 (선택사항)
    df = df.drop('prob', axis=1)
    df.to_csv(f"bs_dsnet_{date_str}_result.csv",index=False)
    
    # F1 score, precision, and recall calculation
    try:
        f1 = f1_score(flattened_label, flattened_prediction, average='binary')
        precision = precision_score(flattened_label, flattened_prediction, average='binary')
        recall = recall_score(flattened_label, flattened_prediction, average='binary')
    except ValueError as e:
        print(f"Error: {e}. Switching to 'macro' average.")
        f1 = f1_score(flattened_label, flattened_prediction, average='macro')
        precision = precision_score(flattened_label, flattened_prediction, average='macro')
        recall = recall_score(flattened_label, flattened_prediction, average='macro')
    
    print("**************************************************")
    print(f'Model Output: {args.inference_model}')
    print(f'F1 score: {f1}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Accuracy: {np.sum(flattened_label == flattened_prediction) / flattened_label.size}') 

20250401


/tmp/ipykernel_1899721/186435780.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(run))


flattened_label shape: (5003745,)
flattened_prediction shape: (5003745,)
Error: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].. Switching to 'macro' average.


/home1/jmt30269/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**************************************************
Model Output: ./results/BS_06_conv2d_unmix_p5_97.02_epoch20.pkl
F1 score: 0.498088582481903
Precision: 0.5317067479726861
Recall: 0.5001350546622082
Accuracy: 0.7726728680218516
20250402


...

**************************************************
Model Output: ./results/BS_06_conv2d_unmix_p5_97.02_epoch20.pkl
F1 score: 0.49572706790578475
Precision: 0.5313172594169809
Recall: 0.49724525499212024
Accuracy: 0.7726249039469437
20250410


/tmp/ipykernel_1899721/186435780.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(run))


flattened_label shape: (5003745,)
flattened_prediction shape: (5003745,)
Error: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].. Switching to 'macro' average.


/home1/jmt30269/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**************************************************
Model Output: ./results/BS_06_conv2d_unmix_p5_97.02_epoch20.pkl
F1 score: 0.4782528280138905
Precision: 0.41296192276864346
Recall: 0.5706479917832067
Accuracy: 0.7695542039012779
